# Using MMIO with PYNQ

## Goal

The aim of this notebook is to show how to use the MMIO (Memory Mapped I/O) PYNQ class.   

## Hardware design

This example uses the same bitstream from the previous example with three AXI GPIO controllers connected to the LEDs, buttons, and switches. While there are PYNQ drivers available to read and write the AXI GPIO LEDs, switches, for demonstration purposes the AXI GPIO controllers will be used to show how the PYNQ MMIO class can be used. 

![AXI GPIO Design](./images/axi_gpio_design.png "AXI GPIO Design")

This notebook will seem very similar to the previous lab. We will be exercising the buttons, switches and LEDs in a similar way,  but you should note that we are using the MMIO class directly, and that there are small differences in code. For the MMIO class, we will be specifying an offset address that we read or write to. 

If you examine the driver code for the *LED*, *switches*, and *buttons* classes, you will notice that they use the PYNQ MMIO class. 

### 1. Download the axi_gpio.bit overlay

In [ ]:
from pynq import Overlay
axi_gpio_design = Overlay("./bitstream/axi_gpio.bit")

Check the IP Dictionary for this design. 

Note the physical memory addresses. They will be used with the MMIO class to map the areas of memory that will be read/written.

In [ ]:
axi_gpio_design.ip_dict

## MMIO class

MMIO can map arrays, or a range of addresses. A physical memory address and a range are required by the MMIO class. 

In this example, the MMIO class will be used to directly access the register space of the AXI GPIO and control the IP. 

An AXI GPIO controller has two channels. In the design, only 1 channel of each AXI controller is used (as described in the previous lab). We will only use two registers:

The data register is mapped to offset 0x0, and the tristate register is mapped to offset 0x4. To use an AXI GPIO, the tri-state register must be set to configure the IP as input or output. The data register can be read or written to. For example, the AXI GPIO connected to the LEDs sets the tri-state register to configure the IP as an output. The LEDs will turn on or off corresponding to the value written to the data register. For the buttons, or switches, the IP is configured as *input* and the value in the data register will be the value corresponding to the position of the switches or buttons. 

In the following example, 3 MMIO instances will be created corresponding  to each AXI GPIO. 

First assign the physical addresses of the controllers to python variables. 

In [ ]:
buttons_address = axi_gpio_design.ip_dict['buttons']['phys_addr']
switches_address = axi_gpio_design.ip_dict['switches']['phys_addr']
leds_address = axi_gpio_design.ip_dict['leds']['phys_addr']

print("Physical address of button: 0x" + format(buttons_address, '02x'))
print("Physical address of button: 0x" + format(switches_address, '02x'))
print("Physical address of button: 0x" + format(leds_address, '02x'))

### 1. Controlling the switches and push-buttons (again!)

An `MMIO` instance is created with an address and optionally a range. The range specifies the range of addresses that can be accessed from the base address. Care must be taken when reading and writing addresses in the system that they physically exist. Reading or writing to location that is not accessible can cause the system to hang.  

In [ ]:
from pynq import MMIO
RANGE = 8 # Number of bytes; 8/4 = 2x 32-bit locations which is all we need for this example
buttons = MMIO(buttons_address, RANGE) 

Write 0xffffffff to the tri-state register at offset 0x4 to configure the IO as inputs.

In [ ]:
buttons.write(0x4, 0xffffffff) 

In [ ]:
print(f"Push-buttons: {buttons.read()}")

As before, try pressing any combination of the push-buttons while rerunning the cell above.

The AXI GPIO controller for the switches can be used in a similar way:

In [ ]:
switches = MMIO(switches_address, RANGE)
switches.write(0x4, 0xffffffff) 

In [ ]:
print(f"Switches: {switches.read()}")

### 2. Controlling the LEDs (again!)

The LEDs can be used in a similar way, this time `0x0` is written to the tri-state register to configure the IO as output. 

In [ ]:
leds = MMIO(leds_address, RANGE)
leds.write(0x4, 0x0) # Write 0x0 to location 0x4; Set tri-state to output

In [ ]:
leds.write(0x0, 0xf) # Write 0xf to location 0x0 (Turn on the LEDs)

### 3. Putting it together (again!)

Similarly to the previous lab, we will run a loop to set the LEDs to the value of the pushbuttons.

Before executing the next cell, make sure Switch 0 (SW0) is "on". While the loop is running, press a push-button and notice the corresponding LED turns on. To exist the loop, change Switch 0 to off.

In [ ]:
while(switches.read(0x0) is 1):
    leds.write(0x0, buttons.read(0x0))